In [150]:
import numpy as np
import pandas as pd
import deeppavlov
import json
import codecs

In [151]:
with codecs.open("../data/train.jsonl", "r", "utf-8") as stream:
    data = stream.readlines()
    data = list(map(json.loads, data))
    print(data[0])

{'idx': 0, 'passage': {'text': '(1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажо

In [152]:
# for i in range(len(data)):
#     data[i]['passage']['text'] = ... #TODO

In [153]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer 
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder
from deeppavlov.models.embedders.tfidf_weighted_embedder import TfidfWeightedEmbedder

In [154]:
model_path = '../data/ft_native_300_ru_wiki_lenta_lemmatize.bin'

tokenizer = RussianTokenizer(lemmas=True, alphas_only=False)

In [155]:
for i in range(len(data)):
    text = data[i]['passage']['text']
    data[i]['passage']['text'] = tokenizer([text])[0]
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        data[i]['passage']['questions'][j]['question'] = tokenizer([question])[0]
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            data[i]['passage']['questions'][j]['answers'][k]['text'] = tokenizer([answer])[0]

In [113]:
model = FasttextEmbedder(model_path)

2020-11-19 17:31:21.383 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/mnt/data/ft_native_300_ru_wiki_lenta_lemmatize.bin`]


In [156]:
def cosine(vec1, vec2):
    return (vec1 * vec2).sum() / np.linalg.norm(vec1) / np.linalg.norm(vec2)

In [157]:
for i in range(len(data)):
    if i % 100 == 0:
        print(i)
    text = data[i]['passage']['text']
    data[i]['passage']['text'] = np.mean(model([text])[0], axis=0)
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        data[i]['passage']['questions'][j]['question'] = np.mean(model([question])[0], axis=0)
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            data[i]['passage']['questions'][j]['answers'][k]['text'] = np.mean(model([answer])[0], axis=0)

0
100
200
300
400


In [158]:
X = []
y = []
for i in range(len(data)):
    text = data[i]['passage']['text']
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            label = data[i]['passage']['questions'][j]['answers'][k]['label']
            row = np.concatenate([text, question, answer])
            X.append(row)
            y.append(label)

In [165]:
with codecs.open("../data/val.jsonl", "r", "utf-8") as stream:
    val = stream.readlines()
    val = list(map(json.loads, val))
    
X_val = []
y_val = []
for i in range(len(val)):
    text = val[i]['passage']['text']
    text = tokenizer([text])[0]
    text = np.mean(model([text])[0], axis=0)
    for j in range(len(val[i]['passage']['questions'])):
        question = val[i]['passage']['questions'][j]['question']
        question = tokenizer([question])[0]
        question = np.mean(model([question])[0], axis=0)
        for k in range(len(val[i]['passage']['questions'][j]['answers'])):
            answer = val[i]['passage']['questions'][j]['answers'][k]['text']
            answer = tokenizer([answer])[0]
            answer = np.mean(model([answer])[0], axis=0)
            row = np.concatenate([text, question, answer])
            X_val.append(row)
            label = val[i]['passage']['questions'][j]['answers'][k]['label']
            y_val.append(label)


In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()

model.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [173]:
predicted = model.predict(X_val)

In [174]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, predicted)

0.6152125279642058

In [175]:
!pip install catboost

     |████████████████████████████████| 66.1 MB 51 kB/s  eta 0:00:011   |▋                               | 1.3 MB 2.9 MB/s eta 0:00:23     |█████████████                   | 26.9 MB 10.5 MB/s eta 0:00:04     |█████████████████▍              | 35.9 MB 12.6 MB/s eta 0:00:03     |███████████████████████▏        | 47.8 MB 1.9 MB/s eta 0:00:10     |█████████████████████████▍      | 52.5 MB 11.0 MB/s eta 0:00:02     |██████████████████████████████  | 62.1 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 7.8 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=577810af27484c211d191cc8ac0fa14dc00873cf649170262c5551af4a6e6d27
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
